<center>
    <h1 style="color:deeppink">Just between us SQL Friends...</h1>
    <h2 style="color:deeppink">RuPaul's Drag Race: Visualized</h2>
    <img src="media/header.gif" height="100px">
</center>

<h2 style="color:deeppink">Import Libraries & Connect SQLite Database</h2>

In [47]:
import pandas as pd
import sqlite3
from IPython.display import Image
from pandas.io.json import json_normalize

drag_race_data_base = sqlite3.connect("drag_race_data.db")

<h2 style="color:deeppink">Define Constants, Global Variables, and API endpoints</h2>

In [48]:
#API URL
DRAG_RACE_API_URL = "http://www.nokeynoshade.party/api/"

#API Endpoints - may not need all of these in the finished product
SEASONS = "seasons"
QUEENS = "queens/all"
SEASON_BY_NUMBER = "seasons/{}"

<h2 style="color:deeppink">Define Functions</h2>

In [174]:
def get_data(endpoint, SEASON = None):
    data = pd.read_json(DRAG_RACE_API_URL + endpoint.format(SEASON))
    return data

def get_seasons():
    df = pd.DataFrame()
    temp_df = pd.DataFrame()
    for i in range(len(season_id_df)):
        temp_df = get_data("seasons/{}", i + 1)
        temp_df = json_normalize(temp_df['queens'])
        temp_df['season_id'] = i + 1
        df = pd.concat([df, temp_df], sort=False)
        i += 1
    return df


<h2 style="color:deeppink">Define SQL Queries</h2>

In [187]:
queens_query = """
    SELECT season_id, queens.id, name, image_url FROM seasons
    INNER JOIN queens ON seasons.id = queens.id
    ORDER BY season_id
"""

queens_seasons_query = """
    SELECT * FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    ORDER BY season_number, place DESC

"""

miss_congeniality_query = """
    SELECT * FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE missCongeniality IS True
    ORDER BY season_id
    
"""

<h2 style="color:deeppink">Define values used for cleanup</h2>

In [188]:
queens_drop_cols = ['queens', 'name']

<h2 style="color:deeppink">Create raw dataframes from API endpoints</h2>

In [189]:
raw_seasons_df = get_data(SEASONS)
raw_queens_df = get_data(QUEENS)

<h2 style="color:deeppink">Clean and organize the data</h2>

In [190]:
# create the season_id_df to look up season numbers by id
season_id_series = raw_seasons_df['id']
season_number_series = raw_seasons_df['seasonNumber']
seasons_dict = {"season_id":season_id_series, "season_number": season_number_series}

season_id_df = pd.DataFrame(seasons_dict)

<h2 style="color:deeppink">Create SQL Tables</h2>

In [191]:
# queens table
raw_queens_df.to_sql('queens', drag_race_data_base, if_exists='replace')

# seasons table
seasons_df = get_seasons().set_index('id').drop(queens_drop_cols, axis=1)
seasons_df.to_sql('seasons', drag_race_data_base, if_exists = 'replace')

# season_ids table
season_id_df.to_sql('season_ids', drag_race_data_base, if_exists='replace')

<h2 style="color:deeppink">Sandbox</h2>
<b style="color:deeppink">Use the space below to experiment. Nothing below will be included in the finished product.</b>

In [198]:
RUN_THIS_QUERY = queens_seasons_query

pd.read_sql(RUN_THIS_QUERY, drag_race_data_base)

,id,place,season_id,index,season_id,season_number,index,id,name,winner,missCongeniality,image_url,quote
0,1.0,9.0,1,0,1,1,0,1,Victoria 'Porkchop' Parker,0,0,http://www.nokeynoshade.party/images/victoria-...,Would you fuck me? I'd fuck me.
1,2.0,8.0,1,0,1,1,1,2,Tammie Brown,0,0,http://www.nokeynoshade.party/images/tammie-br...,"I'm not a slut, I'm a lady. You look under my ..."
2,3.0,7.0,1,0,1,1,2,3,Akashia,0,0,http://www.nokeynoshade.party/images/akashia.jpg,"If I was a girl, I'd be a stripper, or a slut ..."
3,4.0,6.0,1,0,1,1,3,4,Jade,0,0,http://www.nokeynoshade.party/images/jade.jpg,"Jade is definitely my alter-ego, basically she..."
4,5.0,5.0,1,0,1,1,4,5,Ongina,0,0,http://www.nokeynoshade.party/images/ongina.jpg,"If I don't win this, I swear to god I'm gonna ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,46.0,5.0,14,12,14,A4,42,46,Latrice Royale,0,1,http://www.nokeynoshade.party/images/latrice-r...,Jesus is a biscuit! Let him sop you up!
181,104.0,4.0,14,12,14,A4,97,104,Naomi Smalls,0,0,http://www.nokeynoshade.party/images/naomi-sma...,Check your lipstick before you come for me.
182,136.0,3.0,14,12,14,A4,123,136,Monique Heart,0,0,http://www.nokeynoshade.party/images/monique-h...,Cat kitty cat cat cat cat. Cat kitty cat cat c...
183,123.0,1.0,14,12,14,A4,110,123,Trinity Taylor,0,0,http://www.nokeynoshade.party/images/trinity-t...,"I have 50 units of botox in my face, I don't k..."


In [207]:
# Export the test query to a .csv file
pd.read_sql(RUN_THIS_QUERY, drag_race_data_base).to_csv('output.csv', index=True)  